# Student Binary Classification

Determine the classifiers for the student that may lead to a drop out


In [4]:
#update the modules to be used
!pip install matplotlib

In [7]:
#importing the libraries to be used in the 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#import the csv format of the dataset
